# Walmart sales

In [6]:
# Data handling
import pandas as pd

# ML librairies
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import  OneHotEncoder, StandardScaler

# Dataviz librairies
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go

# OS Toolbox
import os

Importing data from the CSV file.

In [7]:
dataset = pd.read_csv(os.getcwd()+"/data/Walmart_Store_sales.csv")
dataset.head(10)

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,6.0,18-02-2011,1572117.54,NaN,59.61,3.045,214.777523,6.858
1,13.0,25-03-2011,1807545.43,0.0,42.38,3.435,128.616064,7.470
2,17.0,27-07-2012,NaN,0.0,NaN,NaN,130.719581,5.936
3,11.0,NaN,1244390.03,0.0,84.57,NaN,214.556497,7.346
4,6.0,28-05-2010,1644470.66,0.0,78.89,2.759,212.412888,7.092
5,4.0,28-05-2010,1857533.70,0.0,NaN,2.756,126.160226,7.896
6,15.0,03-06-2011,695396.19,0.0,69.80,4.069,134.855161,7.658
7,20.0,03-02-2012,2203523.20,0.0,39.93,3.617,213.023622,6.961
8,14.0,10-12-2010,2600519.26,0.0,30.54,3.109,NaN,NaN
9,3.0,NaN,418925.47,0.0,60.12,3.555,224.132020,6.833


Data comprehension :
- Store - the store number
- Date - the week of sales
- Weekly_Sales - sales for the given store
- Holiday_Flag - whether the week is a special holiday week (1 = yes, 0 = no)
- Temperature - Temperature
- Fuel_Price - Cost of fuel
- CPI – Consumer price index
- Unemployment - Unemployment rate

## EDA

### Basic exploration

In [8]:
print("Number of rows : {}".format(dataset.shape[0]))
print("Number of columns : {}".format(dataset.shape[1]))
print()

print("Display of dataset: ")
display(dataset.head())
print()

print("Basics statistics: ")
data_desc = dataset.describe(include="all")
display(data_desc)
print()

print("Data types: ")
display(dataset.dtypes)

print("Percentage of missing values: ")
display(100 * dataset.isnull().sum() / dataset.shape[0])

Number of rows : 150
Number of columns : 8

Display of dataset: 


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,6.0,18-02-2011,1572117.54,NaN,59.61,3.045,214.777523,6.858
1,13.0,25-03-2011,1807545.43,0.0,42.38,3.435,128.616064,7.470
2,17.0,27-07-2012,NaN,0.0,NaN,NaN,130.719581,5.936
3,11.0,NaN,1244390.03,0.0,84.57,NaN,214.556497,7.346
4,6.0,28-05-2010,1644470.66,0.0,78.89,2.759,212.412888,7.092



Basics statistics: 


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,150.000000,132,1.360000e+02,138.000000,132.000000,136.000000,138.000000,135.000000
unique,NaN,85,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,19-10-2012,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN
mean,9.866667,NaN,1.249536e+06,0.079710,61.398106,3.320853,179.898509,7.598430
std,6.231191,NaN,6.474630e+05,0.271831,18.378901,0.478149,40.274956,1.577173
min,1.000000,NaN,2.689290e+05,0.000000,18.790000,2.514000,126.111903,5.143000
25%,4.000000,NaN,6.050757e+05,0.000000,45.587500,2.852250,131.970831,6.597500
50%,9.000000,NaN,1.261424e+06,0.000000,62.985000,3.451000,197.908893,7.470000
75%,15.750000,NaN,1.806386e+06,0.000000,76.345000,3.706250,214.934616,8.150000



Data types: 


Store           float64
Date             object
Weekly_Sales    float64
Holiday_Flag    float64
Temperature     float64
Fuel_Price      float64
CPI             float64
Unemployment    float64
dtype: object

Percentage of missing values: 


Store            0.000000
Date            12.000000
Weekly_Sales     9.333333
Holiday_Flag     8.000000
Temperature     12.000000
Fuel_Price       9.333333
CPI              8.000000
Unemployment    10.000000
dtype: float64

### Preprocessing

Dropping rows with missing values for the target Weekly_Sales :

In [9]:
dataset.dropna(subset="Weekly_Sales", inplace=True)

Dropping rows with outliers for the features Temperature, CPI, Fuel_Price and Unemployment :

In [10]:
drop_outliers_columns_list = [
    "Temperature",
    "CPI",
    "Fuel_Price",
    "Unemployment"
]

for column in drop_outliers_columns_list :

    high_range = dataset[column].mean() + 3 * dataset[column].std()
    low_range = dataset[column].mean() - 3 * dataset[column].std()

    outlier_condition = (dataset[column] > high_range) | (dataset[column] < low_range)
    
    dataset = dataset.loc[~outlier_condition]

Creating usable features Year, Month, Day and DayOfWeek from the Date column, then drop it since we're not going to use it anymore :

In [11]:
dataset["Date"] = pd.to_datetime(dataset["Date"], dayfirst=True)
dataset["Year"] = dataset["Date"].dt.year
dataset["Month"] = dataset["Date"].dt.month
dataset["Day"] = dataset["Date"].dt.day
dataset["DayOfWeek"] = dataset["Date"].dt.dayofweek
dataset.drop("Date", axis=1, inplace=True)

Checking the distribution of date related features :

In [12]:
print(dataset["Year"].value_counts())
print()
print(dataset["Month"].value_counts())
print()
print(dataset["Day"].value_counts())
print()
print(dataset["DayOfWeek"].value_counts())

Year
2010.0    49
2011.0    34
2012.0    30
Name: count, dtype: int64

Month
6.0     14
2.0     12
5.0     12
7.0     12
3.0     10
8.0     10
4.0      9
10.0     8
11.0     8
12.0     7
9.0      6
1.0      5
Name: count, dtype: int64

Day
12.0    10
19.0     7
25.0     7
18.0     6
16.0     6
30.0     6
10.0     6
20.0     5
26.0     5
6.0      4
2.0      4
22.0     4
7.0      4
27.0     4
15.0     3
13.0     3
28.0     3
24.0     3
11.0     3
17.0     3
9.0      2
4.0      2
5.0      2
14.0     2
23.0     2
1.0      2
3.0      2
31.0     1
29.0     1
8.0      1
Name: count, dtype: int64

DayOfWeek
4.0    113
Name: count, dtype: int64


Since the feature DayOfWeek only have an unique value, we drop it :

In [13]:
dataset.drop("DayOfWeek", axis=1, inplace=True)

### Vizualization

In [14]:
fig = px.bar(
    dataset,
    x="Store",
    y="Weekly_Sales",
    title="Sales distributions by store",
    color_discrete_sequence=[px.colors.qualitative.G10[0]]
)

fig.update_layout(
    yaxis_title="Total sales",
    xaxis_title="Store"
)

fig.show()

The total ammount of weekly sales seems very different across stores, it has to be a certain correlation.

In [15]:
fig = px.box(
    dataset,
    x="Month",
    y="Weekly_Sales",
    title="Sales distributions by month",
    color_discrete_sequence=[px.colors.qualitative.G10[0]]
)

fig.update_layout(
    yaxis_title="Sales",
    xaxis_title="Month"
)

fig.show()

It seems that there's maybe a correlation between sales distributions and months, there's is a high variance of the median weekly sales. For example, we can see that in december (i.e. month of Christmas) the median weekly sales is clearly higher than other months.

In [16]:
fig = px.histogram(
    dataset,
    x="Temperature",
    y="Weekly_Sales",
    title="Impact of temperature on sales",
    color_discrete_sequence=[px.colors.qualitative.G10[0]]
)

fig.update_layout(
    yaxis_title="Total sales",
    xaxis_title="Temperature"
)

fig.show()

It seems there is no clear correlation between temperature and weekly sales. Some clusters of higher sales are observed at specific temperature ranges, but these may be influenced by external factors such as holidays (e.g. Christmas) or seasonal demand (e.g.Thanksgiving). So maybe it's not correllated to the temperature itself.

In [17]:
fig = px.histogram(
    dataset,
    x="Fuel_Price",
    y="Weekly_Sales",
    title="Impact of fuel price on sales",
    color_discrete_sequence=[px.colors.qualitative.G10[0]]
)

fig.update_layout(
    yaxis_title="Total sales",
    xaxis_title="Fuel price"
)

fig.show()

There seems to be little to no significant connection between fuel prices and weekly sales. The highest weekly sales values is at a moderate to high fuel prices, but it's difficult to extrapolate a trend since it can be a coincidence with some other external factors.

In [18]:
fig = px.histogram(
    dataset,
    x="CPI",
    y="Weekly_Sales",
    nbins=15,
    title="Impact of inflation (CPI) on sales",
    color_discrete_sequence=[px.colors.qualitative.G10[0]]
)

fig.update_layout(
    yaxis_title="Total sales",
    xaxis_title="CPI"
)

fig.show()

There doesn't appear to be a very clear correlation too, between CPI and weekly sales. The data points seem separate across the range of CPI values. The relationship between CPI and sales might be affected by other factors, such as economic conditions, consumer behavior, or promotional events, rather than CPI alone.

In [19]:
fig = px.histogram(
    dataset,
    x="Unemployment",
    y="Weekly_Sales",
    title="Impact of unemployment rate on sales",
    color_discrete_sequence=[px.colors.qualitative.G10[0]]
)

fig.update_layout(
    yaxis_title="Total sales",
    xaxis_title="Unemployment rate"
)

fig.show()

There doesn't seem to be a strong correlation between unemployment rates and weekly sales. Some of the highest weekly sales values appear to occur at moderate unemployment rates, but this could be coincidental rather than indicative of a trend.

Visualize pairwise dependencies, thanks to a heatmap representation of a correlation matrix (Pearson) :

In [20]:
corr_matrix = dataset.corr().round(2)

fig = ff.create_annotated_heatmap(
    corr_matrix.values,
    x = corr_matrix.columns.tolist(),
    y = corr_matrix.index.tolist()
)

fig.show()

We want to visualize pairwise dependencies, thanks to a heatmap of the correlation matrix. It summarize the observations that we make earlier, but we can also the see a strong correlation between Fuel_Price and Year, which might indicate multicollinearity.

## Baseline model : multivariate linear regression

Separating features from target.

In [21]:
target_variable = "Weekly_Sales"

X = dataset.drop(target_variable, axis=1)
y = dataset[target_variable]

### Train-test split

In [22]:
X_train_unproc, X_test_unproc, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

### Preprocessing : impute, encode and standardize values

In [23]:
numerical_features = ["Temperature", "Fuel_Price", "CPI", "Unemployment", "Year", "Month", "Day"]
categorical_features = ["Store", "Holiday_Flag"]

numerical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="mean")),
        ("scaler", StandardScaler())
    ]
)

categorical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")), 
        ("encoder", OneHotEncoder(drop="first"))
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numerical_transformer, numerical_features),
        ("cat", categorical_transformer, categorical_features)
    ]
)

X_train = preprocessor.fit_transform(X_train_unproc)
X_test = preprocessor.transform(X_test_unproc)

### Train model

In [24]:
linear_regression = LinearRegression()
linear_regression.fit(X_train, y_train)

LinearRegression()

### Performance assessment

In [25]:
data_comparaison = []

print("R2 score on training set : ", linear_regression.score(X_train, y_train))
print("R2 score on test set : ", linear_regression.score(X_test, y_test))
print()

scores = cross_val_score(linear_regression, X_train, y_train, cv=10)
print("The cross-validated accuracy is : ", scores.mean())
print("The standard deviation is : ", scores.std())

data_comparaison.append([
    "Linear",
    "",
    linear_regression.score(X_train, y_train),
    linear_regression.score(X_test, y_test),
    scores.mean(),
    scores.std()
])

R2 score on training set :  0.9731935046590678
R2 score on test set :  0.9344654257543307

The cross-validated accuracy is :  0.9405178371914926
The standard deviation is :  0.04255055442642696


The R2 scores on the training and test sets are fairly high (0.971 and 0.934), but it seems that the model overtfit a bit. Our 10-fold cross-validation shows that the mean accuracy is 0.940 and can vary by $\pm 0.0425$ simply by changing a few examples between the training and test sets. We'll try to reduce overfitting through regularization.

### Features importance

In [26]:
def create_features_importance_df_fig(preprocessor, model_coef) :
    column_names = []

    for name, step, features_list in preprocessor.transformers_:
        if name == 'num':
            features = features_list
        else :
            features = step.get_feature_names_out()
        
        column_names.extend(features)

    df = pd.DataFrame(index=column_names, data=model_coef.transpose()).reset_index()
    df.columns = ["features", "coefficients"]
    df["coefficients"] = df["coefficients"].apply(lambda x : abs(x))
    df = df.sort_values(by = 'coefficients')

    fig = px.bar(
        df,
        x="coefficients",
        y="features"
    )

    fig.update_layout(
        showlegend = False, 
        margin = {'l': 120},
        width=1000,
        height=700
    )

    return df, fig

In [27]:
_, fig = create_features_importance_df_fig(preprocessor, linear_regression.coef_)
fig.show()

Our baseline told us that the store is the most important to predict weekly sales. Beside store, unemployment and CPI seems to have a certain importance too.

## Hyperparameter tuning

In [28]:
ridge = Ridge()

params = {
    "alpha": [0.01, 0.02, 0.03, 0.04, 0.05, 0.6, 0.7]
}

gridsearch_ridge = GridSearchCV(ridge, param_grid=params, cv=10)
gridsearch_ridge.fit(X_train, y_train)

data_comparaison.append([
    "Ridge",
    gridsearch_ridge.best_params_,
    gridsearch_ridge.score(X_train, y_train),
    gridsearch_ridge.score(X_test, y_test),
    gridsearch_ridge.cv_results_["mean_test_score"][gridsearch_ridge.best_index_],
    gridsearch_ridge.cv_results_["std_test_score"][gridsearch_ridge.best_index_]
])

_, fig = create_features_importance_df_fig(preprocessor, gridsearch_ridge.best_estimator_.coef_)
fig.show()

In [29]:
lasso = Lasso()

params = {
    "alpha": [i for i in range(100, 1000, 25)]
}

gridsearch_lasso = GridSearchCV(lasso, param_grid=params, cv=10)
gridsearch_lasso.fit(X_train, y_train)

data_comparaison.append([
    "Lasso",
    gridsearch_lasso.best_params_,
    gridsearch_lasso.score(X_train, y_train),
    gridsearch_lasso.score(X_test, y_test),
    gridsearch_lasso.cv_results_["mean_test_score"][gridsearch_lasso.best_index_],
    gridsearch_lasso.cv_results_["std_test_score"][gridsearch_lasso.best_index_]
])

_, fig = create_features_importance_df_fig(preprocessor, gridsearch_lasso.best_estimator_.coef_)
fig.show()

In [30]:
score_comparaison = pd.DataFrame(data=data_comparaison, columns=["model", "best_hyperparameters", "train_score", "test_score", "cv_mean_score", "cv_mean_std"])
score_comparaison

,model,best_hyperparameters,train_score,test_score,cv_mean_score,cv_mean_std
0,Linear,,0.973194,0.934465,0.940518,0.042551
1,Ridge,{'alpha': 0.02},0.973124,0.937152,0.940656,0.043713
2,Lasso,{'alpha': 450},0.972741,0.943108,0.941990,0.044039


We can't really see a huge difference of performances between those three models, even with regularization. One reason is the size of the dataset himself, which is very small. An other can be the features we have selected : but thanks to L1 regularization (Lasso), which shrink ineficient coefficients (i.e. features) to 0, we can easily see the ones less important to the model. We'll try some features combinaisons, but starting by dropping the 3 less importants regarding Lasso regularization : Year, Holiday_Flag and Day.

### Features selection

In [31]:
target_variable = "Weekly_Sales"
features_to_drop = ["Year", "Holiday_Flag", "Day", "Temperature"]

X = dataset.drop([target_variable, *features_to_drop], axis=1)
y = dataset[target_variable]

X_train_unproc, X_test_unproc, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

numerical_features = ["CPI", "Unemployment", "Month", "Fuel_Price"]
categorical_features = ["Store"]

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numerical_transformer, numerical_features),
        ("cat", categorical_transformer, categorical_features)
    ]
)

X_train = preprocessor.fit_transform(X_train_unproc)
X_test = preprocessor.transform(X_test_unproc)



After few try, it seems that Temperature isn't an importante feature either, so we decide not to use it too.

In [32]:
linear_regression = LinearRegression()
linear_regression.fit(X_train, y_train)

scores = cross_val_score(linear_regression, X_train, y_train, cv=10)

data_comparaison = []

data_comparaison.append([
    "Linear",
    "",
    linear_regression.score(X_train, y_train),
    linear_regression.score(X_test, y_test),
    scores.mean(),
    scores.std()
])

In [33]:
ridge = Ridge()

params = {
    "alpha": [0.008, 0.009, 0.01, 0.02, 0.03, 0.04, 0.05, 0.6, 0.7]
}

gridsearch_ridge = GridSearchCV(ridge, param_grid=params, cv=10)
gridsearch_ridge.fit(X_train, y_train)

data_comparaison.append([
    "Ridge",
    gridsearch_ridge.best_params_,
    gridsearch_ridge.score(X_train, y_train),
    gridsearch_ridge.score(X_test, y_test),
    gridsearch_ridge.cv_results_["mean_test_score"][gridsearch_ridge.best_index_],
    gridsearch_ridge.cv_results_["std_test_score"][gridsearch_ridge.best_index_]
])

In [34]:
lasso = Lasso()

params = {
    "alpha": [i for i in range(100, 1000, 25)]
}

gridsearch_lasso = GridSearchCV(lasso, param_grid=params, cv=10)
gridsearch_lasso.fit(X_train, y_train)

data_comparaison.append([
    "Lasso",
    gridsearch_lasso.best_params_,
    gridsearch_lasso.score(X_train, y_train),
    gridsearch_lasso.score(X_test, y_test),
    gridsearch_lasso.cv_results_["mean_test_score"][gridsearch_lasso.best_index_],
    gridsearch_lasso.cv_results_["std_test_score"][gridsearch_lasso.best_index_]
])

In [35]:
score_comparaison = pd.DataFrame(data=data_comparaison, columns=["model", "best_hyperparameters", "train_score", "test_score", "cv_mean_score", "cv_mean_std"])
score_comparaison

,model,best_hyperparameters,train_score,test_score,cv_mean_score,cv_mean_std
0,Linear,,0.969000,0.928336,0.944151,0.033592
1,Ridge,{'alpha': 0.02},0.968941,0.929988,0.944275,0.034548
2,Lasso,{'alpha': 150},0.968953,0.930425,0.944446,0.034455


Even if we still overfit a little, we have managed to reduce the standard deviation (by 0.01) while maintaining a high average score thanks to regularization and features selection. To go further we would need more data or create new insights by cleaverly combining our features.